# Chapter 16: Natural Language Processing with RNNs and Attention

Pada bab ini, kita membahas pendekatan lanjutan dalam **Natural Language Processing (NLP)**
dengan memanfaatkan **Recurrent Neural Networks (RNN)** dan mekanisme **Attention**.
Pemrosesan bahasa alami memiliki tantangan tersendiri karena bahasa manusia
mengandung konteks, makna implisit, serta ketergantungan jangka panjang antar kata.

Bab ini memperkenalkan transisi penting dari model berbasis urutan klasik
menuju arsitektur modern seperti **Transformer**, yang saat ini menjadi standar
dalam berbagai aplikasi NLP skala besar.


## Tujuan Pembelajaran

Setelah mempelajari bab ini, pembaca diharapkan mampu:
1. Memahami konsep **Char-RNN** untuk generasi teks.
2. Menggunakan **Embedding Layer** dalam tugas analisis sentimen.
3. Menjelaskan arsitektur **Encoder–Decoder** pada terjemahan mesin.
4. Memahami cara kerja **Attention Mechanism**.
5. Mengenali peran **Transformer** dalam NLP modern.


## 1. Menghasilkan Teks Karakter demi Karakter (Char-RNN)

Char-RNN adalah pendekatan generatif yang memprediksi karakter berikutnya
berdasarkan urutan karakter sebelumnya. Model ini tidak memerlukan
pemahaman kata secara eksplisit, melainkan mempelajari pola bahasa
langsung dari urutan karakter.

Pendekatan ini sangat berguna untuk mempelajari struktur bahasa,
meskipun memerlukan data dan waktu pelatihan yang relatif besar
untuk menghasilkan teks yang koheren.


In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

# Tokenisasi tingkat karakter
tokenizer = keras.preprocessing.text.Tokenizer(char_level=True)
sample_text = "To be, or not to be, that is the question."
tokenizer.fit_on_texts([sample_text])

max_id = len(tokenizer.word_index)
encoded_text = tokenizer.texts_to_sequences([sample_text])[0]

print("Jumlah karakter unik:", max_id)
print("Panjang teks:", len(encoded_text))

# Model Char-RNN berbasis GRU
model_char_rnn = keras.models.Sequential([
    keras.layers.GRU(128, return_sequences=True, input_shape=[None, max_id]),
    keras.layers.GRU(128, return_sequences=True),
    keras.layers.TimeDistributed(
        keras.layers.Dense(max_id, activation="softmax")
    )
])

model_char_rnn.compile(
    loss="sparse_categorical_crossentropy",
    optimizer="adam"
)

model_char_rnn.summary()


## 2. Stateful vs Stateless RNN

RNN dapat dikategorikan berdasarkan cara mereka
menangani *hidden state* antar batch:

| Aspek | Stateless RNN | Stateful RNN |
|------|---------------|--------------|
| Hidden state | Di-reset setiap batch | Dipertahankan |
| Urutan data | Dapat diacak | Harus berurutan |
| Konteks | Jangka pendek | Jangka panjang |

Stateful RNN sangat efektif untuk data berurutan panjang,
namun membutuhkan pengelolaan data yang lebih ketat
selama proses pelatihan.


In [ ]:
## 3. Analisis Sentimen

Analisis sentimen bertujuan untuk menentukan opini
positif atau negatif dari sebuah teks. Dalam NLP modern,
kata-kata diubah menjadi vektor padat menggunakan
**Embedding Layer**, yang mampu merepresentasikan
hubungan semantik antar kata.

Penggunaan **Masking** memastikan bahwa token padding
tidak mempengaruhi proses pembelajaran model.


In [ ]:
model_sentiment = keras.models.Sequential([
    keras.layers.Embedding(
        input_dim=10000,
        output_dim=128,
        mask_zero=True
    ),
    keras.layers.GRU(128, return_sequences=True),
    keras.layers.GRU(128),
    keras.layers.Dense(1, activation="sigmoid")
])

model_sentiment.compile(
    loss="binary_crossentropy",
    optimizer="adam",
    metrics=["accuracy"]
)

model_sentiment.summary()


## 4. Encoder–Decoder untuk Terjemahan Mesin

Model Encoder–Decoder terdiri dari dua komponen utama:
Encoder memproses kalimat sumber dan mengubahnya
menjadi representasi vektor, sedangkan Decoder
menghasilkan kalimat target secara bertahap.

Keterbatasan utama pendekatan ini adalah
ketergantungan pada satu vektor konteks,
yang menjadi kurang efektif untuk kalimat panjang.


## 5. Mekanisme Attention

Attention memungkinkan model untuk memfokuskan
perhatiannya pada bagian tertentu dari input
yang paling relevan saat menghasilkan output.

Dengan attention, decoder tidak lagi bergantung
pada satu vektor ringkasan, melainkan dapat
mengakses seluruh output encoder.


## 6. Transformer

Transformer adalah arsitektur yang sepenuhnya
mengandalkan self-attention tanpa menggunakan RNN.
Pendekatan ini memungkinkan pelatihan yang
lebih cepat dan skalabel.

Komponen utama Transformer:
- Multi-Head Attention
- Positional Encoding
- Residual Connections


In [ ]:
# Contoh Multi-Head Attention
mha_layer = keras.layers.MultiHeadAttention(num_heads=8, key_dim=64)

dummy_input = tf.random.uniform((32, 50, 512))
attention_output = mha_layer(dummy_input, dummy_input)

print("Output shape:", attention_output.shape)


## 7. Model Bahasa Modern

BERT dan GPT adalah contoh model bahasa modern
berbasis Transformer.

- **BERT** unggul dalam pemahaman konteks dua arah.
- **GPT** unggul dalam generasi teks secara autoregresif.

Keduanya menjadi fondasi utama dalam berbagai
aplikasi NLP modern.


## Ringkasan Bab

1. NLP memerlukan model yang mampu memahami konteks urutan.
2. Char-RNN memungkinkan generasi teks berbasis karakter.
3. Embedding dan Attention adalah komponen kunci NLP modern.
4. Transformer telah menggantikan RNN pada banyak tugas NLP.
